In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !%cd /content/drive/MyDrive/zeroshot_new_paper

In [36]:
# import sys
# sys.path.insert(0,'/content/drive/MyDrive/zeroshot_new_paper')

Found cached dataset ag_news (/Users/alealcoforado/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


# === KeyZeroBERTo ===

# Dependencies

In [37]:
# !pip install transformers
# !pip install keybert==0.5.0
# !pip install datasets
# !pip install flair|
# %env PYTORCH_ENABLE_MPS_FALLBACK=1


In [38]:
# !pip install pandarallel

In [39]:
# !pip install evaluate

In [364]:
%load_ext autoreload
%autoreload 2
import nltk
import ipywidgets
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from transformers import pipeline
import torch
import datetime
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK']="1"
import datasets_handler
import evaluation_metrics
import zeroberto
# def getAgora():
#     return str(datetime.datetime.now().strftime("%Y_%m_%d__%H_%M_%S"))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Data Prep

## Source

In [417]:
which_dataset = 'folhauol' 
# which_dataset = 'bbc-news'
# which_dataset = 'ag_news'

hyp_template = "{}."
# hyp_template = "O tema principal deste texto é {}."
# hyp_template = "this text is about {}."
# hyp_template = "this article is about {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)


## Dataset build

### Train Data

In [418]:
train = raw_data
len(train),train.columns

(92241,
 Index(['Unnamed: 0', 'title', 'text', 'date', 'category', 'subcategory',
        'link', 'data', 'full_text'],
       dtype='object'))

In [419]:
labels_list = list(train[class_col].unique())
print(labels_list)

['Poder e Política no Brasil', 'Mercado', 'Notícias de fora do Brasil', 'Esporte', 'Tecnologia', 'Meio Ambiente', 'Equilíbrio e Saúde', 'Educação', 'TV, Televisão e Entretenimento', 'Ciência', 'Turismo', 'Comida']


# Sentence Extraction

In [420]:
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()

In [421]:
train['sentences'] = train[data_col].progress_apply(lambda txt:tokenizer.tokenize(txt))
train.head(2)   

100%|██████████| 92241/92241 [00:16<00:00, 5556.81it/s]


Unnamed: 0                                              title  \
0           0  Lula diz que está 'lascado', mas que ainda tem...   
1           2  Três reportagens da Folha ganham Prêmio Petrob...   

                                                text        date  \
0  Com a possibilidade de uma condenação impedir ...  2017-09-10   
1  Três reportagens da Folha foram vencedoras do ...  2017-09-10   

                     category subcategory  \
0  Poder e Política no Brasil         NaN   
1  Poder e Política no Brasil         NaN   

                                                link  \
0  http://www1.folha.uol.com.br/poder/2017/10/192...   
1  http://www1.folha.uol.com.br/poder/2017/10/192...   

                                                data  \
0  Lula diz que está 'lascado', mas que ainda tem...   
1  Três reportagens da Folha ganham Prêmio Petrob...   

                                           full_text  \
0  Lula diz que está 'lascado', mas que ainda tem...   
1  Três reportagens da Folha ganham Prêmio Petrob...   

                                           sentences  
0  [Lula diz que está 'lascado', mas que ainda te...  
1  [Três reportagens da Folha ganham Prêmio Petro...

In [422]:
sentences_to_classify = []
for sentences in train['sentences']:
  try:
    for sentence in sentences:
      sentences_to_classify.append(sentence)
      break
  except:
    print(train['sentences'])
train['sentence_to_zeroshot'] = sentences_to_classify
len(train)


92241

In [423]:
train['len_sentences'] = train['sentences'].progress_apply(lambda txt:len(txt))
# train[['len_sentences']].describe()

100%|██████████| 92241/92241 [00:00<00:00, 1564867.13it/s]


# Zero-Shot

## Parameters

In [424]:
max_inferences = len(train) #estava em 5000
# max_inferences = 50000

# zeroshot_method = "probability_threshold"
probability_goal = 0.9

zeroshot_method = "top_n_goal"  
top_n_goal = 8

zeroshot_config = {
    'dataset':which_dataset,
    'method':zeroshot_method,
    'prob_goal':probability_goal,
    'top_n_goal':top_n_goal,
    'max_inferences':max_inferences,
    'labels':labels_list,#list(dict_classes_folha.values())
    'template': hyp_template,
    'random_state':422
}
zeroshot_config

{'dataset': 'folhauol',
 'method': 'top_n_goal',
 'prob_goal': 0.9,
 'top_n_goal': 8,
 'max_inferences': 92241,
 'labels': ['Poder e Política no Brasil',
  'Mercado',
  'Notícias de fora do Brasil',
  'Esporte',
  'Tecnologia',
  'Meio Ambiente',
  'Equilíbrio e Saúde',
  'Educação',
  'TV, Televisão e Entretenimento',
  'Ciência',
  'Turismo',
  'Comida'],
 'template': '{}.',
 'random_state': 422}

## Data

In [425]:
zeroshot_data = train.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state'])
len(zeroshot_data)

92241

## Model

In [426]:
# classifier = pipeline("zero-shot-classification",
#                       model="joeddav/xlm-roberta-large-xnli",device="cpu") # device = 'cuda'  ##X## DESCOMENTAR AQUI 

classifier = zeroberto.ZeroBERTo(classes_list=zeroshot_config['labels'],hypothesis_template=zeroshot_config['template'])  ##X## COMENTAR AQUI 

## Inference

In [427]:
%%time
# zeroshot_results = zeroberto.runZeroShotPipeline(classifier,zeroshot_data['sentence_to_zeroshot'],zeroshot_config)  ##X## DESCOMENTAR AQUI 

zeroshot_results = zeroberto.runZeroberto(classifier,zeroshot_data[data_col],zeroshot_config)  ##X## COMENTAR AQUI 
# print(len(zeroshot_results))
### 100 predições ---> <240 segundos (4 min)
### 200 predições ---> <480 segundos (8 min)
### 1000 predições ---> <2400 segundos (40 min)
### 5000 predições ---> 12000 segundos (200 min) (3h 20min)

Preds: 50  - Total time: 2.83 seconds - ETA: 86.9 minutes
Preds: 100  - Total time: 5.61 seconds - ETA: 86.3 minutes
Preds: 150  - Total time: 8.37 seconds - ETA: 85.8 minutes
Preds: 200  - Total time: 11.11 seconds - ETA: 85.4 minutes
Preds: 250  - Total time: 13.84 seconds - ETA: 85.1 minutes
Preds: 300  - Total time: 16.55 seconds - ETA: 84.8 minutes
Preds: 350  - Total time: 19.28 seconds - ETA: 84.7 minutes
Preds: 400  - Total time: 22.03 seconds - ETA: 84.7 minutes
Preds: 450  - Total time: 24.9 seconds - ETA: 85.1 minutes
Preds: 500  - Total time: 27.65 seconds - ETA: 85.0 minutes
Preds: 550  - Total time: 30.38 seconds - ETA: 84.9 minutes
Preds: 600  - Total time: 33.12 seconds - ETA: 84.9 minutes
Preds: 650  - Total time: 35.84 seconds - ETA: 84.8 minutes
Preds: 700  - Total time: 38.58 seconds - ETA: 84.7 minutes
Preds: 750  - Total time: 41.29 seconds - ETA: 84.6 minutes
Preds: 800  - Total time: 44.02 seconds - ETA: 84.6 minutes
Preds: 850  - Total time: 46.76 seconds - ETA

In [ ]:
label_probabilities_df = pd.DataFrame(zeroshot_results,columns=zeroshot_config['labels'])  ##X## COMENTAR AQUI 

In [ ]:
# label_probabilities_df = zeroberto.formatZeroShotResults(zeroshot_results) ##X## DESCOMENTAR AQUI 

In [ ]:
def getTopLabel(row):
  return row.idxmax()
def getTopProb(row):
  return row.max()

label_results = label_probabilities_df.apply(getTopLabel,axis=1)
prob_results = label_probabilities_df.apply(getTopProb,axis=1)
label_results_df = pd.Series(label_results,name='prediction')

In [ ]:
true_labels_df = raw_data[raw_data.index.isin(train.index)][class_col]
final_result_df = pd.concat([true_labels_df,label_results_df],axis=1)

In [ ]:
vector_true = ( final_result_df[class_col].str.strip().str.lower() 
               == final_result_df['prediction'].str.strip().str.lower() )

## Metrics

In [ ]:
final_result_df_encoded = evaluation_metrics.Encoder(final_result_df,['prediction',class_col])

In [ ]:
try:print(all_metrics)
except:pass
all_metrics = evaluation_metrics.get_metrics(final_result_df_encoded['prediction_code'][0:len(zeroshot_results)].to_list(),
                               final_result_df_encoded[class_col+'_code'][0:len(zeroshot_results)].to_list())
print(all_metrics)

{'weighted': [{'accuracy': 0.24987460815047022}, {'precision': 0.24943974820186196}, {'recall': 0.24987460815047022}, {'f1': 0.2407319010097514}], 'macro': [{'accuracy': 0.24987460815047022}, {'precision': 0.24943974820186196}, {'recall': 0.24987460815047025}, {'f1': 0.2407319010097514}]}
{'weighted': [{'accuracy': 0.1562645678169144}, {'precision': 0.19438937509474985}, {'recall': 0.1562645678169144}, {'f1': 0.14217481171296534}], 'macro': [{'accuracy': 0.1562645678169144}, {'precision': 0.08357413434749388}, {'recall': 0.08362697941972495}, {'f1': 0.06269830263900834}]}


### Probabilities

In [ ]:
df_predictions_probs = pd.concat([final_result_df_encoded,
                                  pd.Series(prob_results,name='top_probability'),
                                  pd.Series(vector_true,name='right_wrong')],axis=1)

##### Top N results


In [ ]:
def get_top_n_results(dataframe_results,top_n=1):
    df_top_n = dataframe_results.sort_values(['top_probability','prediction'], ascending=False).groupby('prediction').head(top_n)

    accuracy_top_n = df_top_n.right_wrong.sum()/len(df_top_n.right_wrong)

    accuracy = { "accuracy in top {}".format(top_n) : round(accuracy_top_n,3)}
    print(accuracy)
    return 

get_top_n_results(df_predictions_probs,top_n = 1)
get_top_n_results(df_predictions_probs,top_n = 2)
get_top_n_results(df_predictions_probs,top_n = 3)
get_top_n_results(df_predictions_probs,top_n = 4)
get_top_n_results(df_predictions_probs,top_n = 5)
get_top_n_results(df_predictions_probs,top_n = 6)
get_top_n_results(df_predictions_probs,top_n = 7)
get_top_n_results(df_predictions_probs,top_n = 8)
get_top_n_results(df_predictions_probs,top_n = 9)
get_top_n_results(df_predictions_probs,top_n = 10)
get_top_n_results(df_predictions_probs,top_n = 11)
get_top_n_results(df_predictions_probs,top_n = 12)
get_top_n_results(df_predictions_probs,top_n = 13)
get_top_n_results(df_predictions_probs,top_n = 14)
get_top_n_results(df_predictions_probs,top_n = 15)
get_top_n_results(df_predictions_probs,top_n = 16)
get_top_n_results(df_predictions_probs,top_n = len(zeroshot_results))

{'accuracy in top 1': 0.0}
{'accuracy in top 2': 0.125}
{'accuracy in top 3': 0.111}
{'accuracy in top 4': 0.104}
{'accuracy in top 5': 0.133}
{'accuracy in top 6': 0.125}
{'accuracy in top 7': 0.119}
{'accuracy in top 8': 0.104}
{'accuracy in top 9': 0.111}
{'accuracy in top 10': 0.117}
{'accuracy in top 11': 0.114}
{'accuracy in top 12': 0.104}
{'accuracy in top 13': 0.103}
{'accuracy in top 14': 0.101}
{'accuracy in top 15': 0.094}
{'accuracy in top 16': 0.089}
{'accuracy in top 92241': 0.156}


## Save Results

In [406]:
evaluation_metrics.saveZeroshotResults(zeroshot_config,df_predictions_probs)

predictions_and_probabilities_test_2023_02_13__17_32_30.csv
zeroshot_config_test_2023_02_13__17_32_30.csv


'2023_02_13__17_32_30'

# Load Previous Results

In [30]:
# df_test_results = pd.read_csv(r'/Users/alealcoforado/Documents/Projetos/Datasets/predictions_and_probabilities_test1.csv')

In [31]:
# df_test_results.index = df_test_results['Unnamed: 0']
# df_test_results.drop(columns="Unnamed: 0",inplace=True)